# Operações entre Datraframes e Armazenamentos

In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [2]:
sc = SparkContext("local[*]", "postech").getOrCreate()
spark = SparkSession.builder \
      .master("local[*]") \
      .appName("postech") \
      .getOrCreate()

## Transformações vs Ações

No Apache Spark, as operações são divididas em duas categorias principais: transformações e ações. As transformações, como map, filter e join, são usadas para criar novos RDDs a partir dos existentes. Elas são avaliadas de forma preguiçosa, o que significa que o Spark não executa a transformação imediatamente, mas sim monta um plano de execução que será realizado mais tarde. Estas operações não alteram os RDDs originais, preservando a imutabilidade.

As ações, por outro lado, como collect, count, e saveAsTextFile, disparam a execução das transformações acumuladas. Elas produzem um resultado final, seja retornando dados ao programa driver ou salvando-os em um sistema de armazenamento externo. As ações são essenciais para obter resultados concretos a partir do conjunto de transformações definidas em um programa Spark.

In [11]:
data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 2, 4, 6, 8]
rdd = sc.parallelize(data)

rdd_map = rdd.map(lambda x: x*2)
rdd_filter = rdd.filter(lambda x: x%2==0)
rdd_distinct = rdd.distinct()

Para "executar" o plano criado usamos a função collect()

In [9]:
rdd_map.collect()

[2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 4, 8, 12, 16]

In [10]:
rdd_filter.collect()

[2, 4, 6, 8, 10, 2, 4, 6, 8]

In [12]:
rdd_distinct.collect()

[8, 1, 9, 2, 10, 3, 4, 5, 6, 7]

In [19]:
data = [('A', 1), ('B', 2), ('A', 3), ('A', 4), ('C', 5)]
rdd = sc.parallelize(data)

rdd_group = rdd.groupByKey()
rdd_reduce = rdd.reduceByKey(lambda x, y: x+y)
rdd_sort = rdd.sortByKey()

In [16]:
rdd_group.mapValues(list).collect()

[('B', [2]), ('C', [5]), ('A', [1, 3, 4])]

In [18]:
rdd_reduce.collect()

[('B', 2), ('C', 5), ('A', 8)]

In [20]:
rdd_sort.collect()

[('A', 1), ('A', 3), ('A', 4), ('B', 2), ('C', 5)]

In [21]:
data_A = [1, 2, 3, 4]
data_B = [3, 4, 5, 6]
rdd_A = sc.parallelize(data_A)
rdd_B = sc.parallelize(data_B)

rdd_union = rdd_A.union(rdd_B)

In [22]:
rdd_union.collect()

[1, 2, 3, 4, 3, 4, 5, 6]

Existem outras funções que funcionam como "actions"

In [24]:
rdd_union.union(rdd_B).count()

12

In [25]:
rdd_union.reduce(lambda x, y: x+y)

28

In [35]:
acc = sc.accumulator(0)
rdd_union.foreach(lambda x: acc.add(x))
print(acc)

28


In [32]:
rdd_union.countByValue()

defaultdict(int, {1: 1, 2: 1, 3: 2, 4: 2, 5: 1, 6: 1})

In [33]:
rdd_union.take(3)

[1, 2, 3]

In [36]:
rdd_union.top(3)

[6, 5, 4]